# Setup

In [ ]:
pip install traker

In [5]:
from trak import TRAKer

def get_trak_matrix(
    train_dl, val_dl, model, ckpts, train_set_size, val_set_size, **kwargs
):
    if kwargs is None or kwargs.get("task") is None:
        task = "image_classification"
    else:
        task = kwargs.pop("task")

    traker = TRAKer(model=model, task=task, train_set_size=train_set_size, **kwargs)

    for model_id, checkpoint in enumerate(ckpts):
        traker.load_checkpoint(checkpoint, model_id=model_id)
        for batch in train_dl:
            batch = [x.cuda() for x in batch]
            # batch should be a tuple/list of inputs and labels
            traker.featurize(batch=batch, num_samples=batch[0].shape[0])

    traker.finalize_features()

    for model_id, checkpoint in enumerate(ckpts):
        traker.start_scoring_checkpoint(
            exp_name="test",
            checkpoint=checkpoint,
            model_id=model_id,
            num_targets=val_set_size,
        )
    for batch in val_dl:
        batch = [x.cuda() for x in batch]
        traker.score(batch=batch, num_samples=batch[0].shape[0])

    scores = traker.finalize_scores(exp_name="test")
    return scores


In [6]:
import torch
import numpy as np
from torch.nn import functional as F

class DDA:
    def __init__(
        self,
        model,
        checkpoints,
        train_dataloader,
        val_dataloader,
        group_indices,
        train_set_size=None,
        val_set_size=None,
        trak_scores=None,
        trak_kwargs=None,
        device="cuda",
    ) -> None:
        
        self.model = model
        self.checkpoints = checkpoints
        self.dataloaders = {"train": train_dataloader, "val": val_dataloader}
        self.group_indices = group_indices
        self.device = device

        if trak_scores is not None:
            self.trak_scores = trak_scores
        else:
            try:
                self.train_set_size = len(train_dataloader.dataset)
                self.val_set_size = len(val_dataloader.dataset)
            except AttributeError as e:
                print(
                    f"No dataset attribute found in train_dataloader or val_dataloader. {e}"
                )
                if train_set_size is None or val_set_size is None:
                    raise ValueError(
                        "train_set_size and val_set_size must be specified if "
                        "train_dataloader and val_dataloader do not have a "
                        "dataset attribute."
                    ) from e
                self.train_set_size = train_set_size
                self.val_set_size = val_set_size

            # Step 1: compute TRAK scores
            if trak_kwargs is not None:
                trak_scores = get_trak_matrix(
                    train_dl=self.dataloaders["train"],
                    val_dl=self.dataloaders["val"],
                    model=self.model,
                    ckpts=self.checkpoints,
                    train_set_size=self.train_set_size,
                    val_set_size=self.val_set_size,
                    **trak_kwargs,
                )
            else:
                trak_scores = get_trak_matrix(
                    train_dl=self.dataloaders["train"],
                    val_dl=self.dataloaders["val"],
                    model=self.model,
                    ckpts=self.checkpoints,
                    train_set_size=self.train_set_size,
                    val_set_size=self.val_set_size,
                )

            self.trak_scores = trak_scores

    def get_group_losses(self, model, val_dl, group_indices) -> list:
        losses = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_dl:
                outputs = model(inputs.to(self.device))
                loss = F.cross_entropy(
                    outputs, labels.to(self.device), reduction="none"
                )
                losses.append(loss)
        losses = torch.cat(losses)

        n_groups = len(set(group_indices))
        group_losses = [losses[group_indices == i].mean() for i in range(n_groups)]
        return group_losses

    def compute_group_alignment_scores(self, trak_scores, group_indices, group_losses):
        n_groups = len(set(group_indices))
        S = np.array(trak_scores)
        g = [
            group_losses[i].cpu().numpy() * S[:, np.array(group_indices) == i].mean(axis=1)
            for i in range(n_groups)
        ]
        g = np.stack(g)
        group_alignment_scores = g.mean(axis=0)
        return group_alignment_scores

    def get_debiased_train_indices(
        self, group_alignment_scores, use_heuristic=True, num_to_discard=None
    ):
        if use_heuristic:
            return [i for i, score in enumerate(group_alignment_scores) if score >= 0]

        if num_to_discard is None:
            raise ValueError("num_to_discard must be specified if not using heuristic.")

        sorted_indices = sorted(
            range(len(group_alignment_scores)),
            key=lambda i: group_alignment_scores[i],
        )
        return sorted_indices[num_to_discard:]

    def debias(self, use_heuristic=True, num_to_discard=None):
        group_losses = self.get_group_losses(
            model=self.model,
            val_dl=self.dataloaders["val"],
            group_indices=self.group_indices,
        )

        group_alignment_scores = self.compute_group_alignment_scores(
            self.trak_scores, self.group_indices, group_losses
        )
        
        debiased_train_inds = self.get_debiased_train_indices(
            group_alignment_scores,
            use_heuristic=use_heuristic,
            num_to_discard=num_to_discard,
        )

        return debiased_train_inds


In [7]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# CelebA

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm

celeba_images_path = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"
partition_file = "/kaggle/input/celeba-dataset/list_eval_partition.csv"
attributes_file = "/kaggle/input/celeba-dataset/list_attr_celeba.csv"

partitions = pd.read_csv(partition_file)
attributes = pd.read_csv(attributes_file)

def get_dataloader(
        batch_size=128, num_workers=4, split="train", shuffle=False, augment=True
    ):
    if augment:
        transforms_pipeline = transforms.Compose(
            [
                transforms.RandomHorizontalFlip(),
                transforms.CenterCrop(178),
                transforms.Resize(128),
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
            ]
        )
    else:
        transforms_pipeline = transforms.Compose(
            [
                transforms.CenterCrop(178),
                transforms.Resize(128),
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
            ]
        )

    attributes.iloc[:, 1:] = attributes.iloc[:, 1:].map(lambda x: 1 if x == 1 else 0)

    total_indices = len(attributes)
    reduced_indices = np.random.choice(attributes.index, size=total_indices // 5, replace=False)

    blond_indices = attributes[attributes["Blond_Hair"] == 1].index.intersection(reduced_indices)
    non_blond_indices = attributes[attributes["Blond_Hair"] == 0].index.intersection(reduced_indices)

    num_non_blond = len(non_blond_indices)
    num_blond = min(len(blond_indices), num_non_blond * 4)
    selected_blond_indices = np.random.choice(blond_indices, num_blond, replace=False)
    selected_indices = np.concatenate([selected_blond_indices, non_blond_indices])

    dataset_split = "train" if split == "train" else "valid"
    if dataset_split == "train":
        selected_indices = partitions[
            (partitions["partition"] == 0) & partitions.index.isin(selected_indices)
        ].index
    else:
        selected_indices = partitions[
            (partitions["partition"] == 1) & partitions.index.isin(selected_indices)
        ].index

    class CelebADataset(torch.utils.data.Dataset):
        def __init__(self, indices, img_dir, attributes, transform=None):
            self.indices = indices
            self.img_dir = img_dir
            self.attributes = attributes
            self.transform = transform

        def __len__(self):
            return len(self.indices)

        def __getitem__(self, idx):
            img_index = self.indices[idx]
            img_name = self.attributes.iloc[img_index, 0]
            img_path = os.path.join(self.img_dir, img_name)

            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)

            label = torch.tensor(self.attributes.iloc[img_index]["Blond_Hair"], dtype=torch.long)
            return image, label

    dataset = CelebADataset(
        indices=selected_indices,
        img_dir=celeba_images_path,
        attributes=attributes,
        transform=transforms_pipeline
    )

    loader = DataLoader(
        dataset=dataset, shuffle=shuffle, batch_size=batch_size, num_workers=num_workers
    )

    return loader, dataset

from torchvision.models import resnet18, ResNet18_Weights
model_before_mitigating = resnet18(weights=ResNet18_Weights.DEFAULT)
model_before_mitigating.fc = nn.Linear(model_before_mitigating.fc.in_features, 2)
model_before_mitigating = model_before_mitigating.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_before_mitigating.parameters(), lr=0.001)

train_loader, train_dataset = get_dataloader(batch_size=32, split="train", shuffle=True)
val_loader, val_dataset = get_dataloader(batch_size=32, split="val", shuffle=False, augment=False)

num_epochs = 1
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    model_before_mitigating.train()
    epoch_loss = 0.0
    for images, labels in tqdm(train_loader, desc="Training"):
        images = images.cuda()
        labels = labels.cuda()

        outputs = model_before_mitigating(images)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss = epoch_loss / len(train_loader)
    print(f"Training Loss: {avg_loss:.4f}")

    model_before_mitigating.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation"):
            images = images.cuda()
            labels = labels.cuda()
            outputs = model_before_mitigating(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")

print("Training and evaluation completed.")

In [7]:
from sklearn.metrics import accuracy_score
import numpy as np

def evaluate_worst_group_accuracy(model, val_loader, group_inds, device="cuda"):
    model.eval()
    group_preds = {i: [] for i in set(group_inds)}
    group_labels = {i: [] for i in set(group_inds)}

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(tqdm(val_loader, desc="Evaluating WGA")):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            batch_start = batch_idx * val_loader.batch_size
            batch_end = batch_start + len(labels)
            batch_groups = group_inds[batch_start:batch_end]

            for i, group in enumerate(batch_groups):
                group_preds[group].append(preds[i])  
                group_labels[group].append(labels.cpu().numpy()[i])  

    group_accuracies = {}
    for group in group_preds.keys():
        if len(group_preds[group]) == 0 or len(group_labels[group]) == 0:
            group_accuracies[group] = 0.0
            continue

        preds = np.array(group_preds[group])
        truths = np.array(group_labels[group])
        group_accuracies[group] = accuracy_score(truths, preds)

    for group, acc in group_accuracies.items():
        print(f"Group {group} Accuracy: {acc:.4f}")

    worst_group_accuracy = min(group_accuracies.values())
    return worst_group_accuracy, group_accuracies

In [8]:
attributes = pd.read_csv(attributes_file)
attributes.iloc[:, 1:] = attributes.iloc[:, 1:].map(lambda x: 1 if x == 1 else 0)

def define_subgroups(row):
    if row["Young"] == 1 and row["Male"] == 1:
        return "young-male"
    elif row["Young"] == 1 and row["Male"] == 0:
        return "young-female"
    elif row["Young"] == 0 and row["Male"] == 1:
        return "old-male"
    elif row["Young"] == 0 and row["Male"] == 0:
        return "old-female"

attributes["subgroup"] = attributes.apply(define_subgroups, axis=1)

subgroup_mapping = {name: i for i, name in enumerate(sorted(attributes["subgroup"].unique()))}
attributes["group_index"] = attributes["subgroup"].map(subgroup_mapping)

val_indices = list(val_loader.dataset.indices)
group_labels = attributes.loc[attributes.index.intersection(val_indices), "group_index"].values
group_inds = attributes['subgroup'].map(subgroup_mapping).values # I should check the group indices later to make sure it works well or not.


print("Subgroup Distribution in Validation Set:")
print(attributes.loc[val_indices, "subgroup"].value_counts())
# print(f"Sample Group Indices: {group_inds[:10]}")

print("Unique Subgroups in Validation Set:")
print(attributes.loc[val_indices, "subgroup"].unique())

Subgroup Distribution in Validation Set:
subgroup
young-female    1970
young-male      1077
old-male         632
old-female       359
Name: count, dtype: int64
Unique Subgroups in Validation Set:
['young-female' 'old-male' 'young-male' 'old-female']


**Calculating Fairness Metrics for Young and Old Groups**

In [9]:
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

def evaluate_group_accuracies(model, val_loader, group_labels, device="cuda"):
    model.eval()
    group_preds = {g: [] for g in set(group_labels)}
    group_truths = {g: [] for g in set(group_labels)}

    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(val_loader, desc="Evaluating Group Accuracies")):
            images = images.to(device)
            labels = labels.to(device)  

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            batch_start = i * val_loader.batch_size
            batch_end = batch_start + len(labels)
            batch_groups = group_labels[batch_start:batch_end]

            for j, group in enumerate(batch_groups):
                group_preds[group].append(preds[j])
                group_truths[group].append(labels.cpu().numpy()[j])

    group_accuracies = {}
    for group in group_preds:
        if len(group_preds[group]) == 0:
            group_accuracies[group] = 0.0
        else:
            preds = np.array(group_preds[group])
            truths = np.array(group_truths[group])
            group_accuracies[group] = accuracy_score(truths, preds)

    for group, acc in group_accuracies.items():
        print(f"Group {group} Accuracy: {acc:.4f}")
    
    return group_accuracies

def evaluate_demographic_parity(model, val_loader, group_labels, device="cuda"):
    model.eval()
    group_pprs = {g: [] for g in set(group_labels)}

    with torch.no_grad():
        for i, (images, _) in enumerate(tqdm(val_loader, desc="Evaluating Demographic Parity")):
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            batch_start = i * val_loader.batch_size
            batch_end = batch_start + len(preds)
            batch_groups = group_labels[batch_start:batch_end]

            for j, group in enumerate(batch_groups):
                group_pprs[group].append(preds[j])

    ppr_disparities = {}
    for group in group_pprs:
        group_positive_rate = np.mean(group_pprs[group])
        ppr_disparities[group] = group_positive_rate

    for group, ppr in ppr_disparities.items():
        print(f"Group {group} PPR: {ppr:.4f}")
    
    return ppr_disparities

def evaluate_equal_opportunity(model, val_loader, group_labels, device="cuda"):
    model.eval()
    group_tprs = {g: [] for g in set(group_labels)}

    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(val_loader, desc="Evaluating Equal Opportunity")):
            images = images.to(device)
            labels = labels.to(device)  

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            batch_start = i * val_loader.batch_size
            batch_end = batch_start + len(labels)
            batch_groups = group_labels[batch_start:batch_end]

            for j, group in enumerate(batch_groups):
                tp = (preds[j] == 1 and labels[j].cpu().numpy() == 1)
                actual_positive = labels[j].cpu().numpy() == 1
                group_tprs[group].append(tp / (actual_positive + 1e-8)) 

    tpr_disparities = {}
    for group in group_tprs:
        tpr_disparities[group] = np.mean(group_tprs[group])

    for group, tpr in tpr_disparities.items():
        print(f"Group {group} TPR: {tpr:.4f}")
    
    return tpr_disparities

def evaluate_equalized_odds(model, val_loader, group_labels, device="cuda"):
    model.eval()
    group_tprs = {g: [] for g in set(group_labels)}
    group_fprs = {g: [] for g in set(group_labels)}

    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(val_loader, desc="Evaluating Equalized Odds")):
            images = images.to(device)
            labels = labels.to(device) 

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()

            batch_start = i * val_loader.batch_size
            batch_end = batch_start + len(labels)
            batch_groups = group_labels[batch_start:batch_end]

            for j, group in enumerate(batch_groups):
                tp = (preds[j] == 1 and labels[j].cpu().numpy() == 1)
                fp = (preds[j] == 1 and labels[j].cpu().numpy() == 0)
                actual_positive = labels[j].cpu().numpy() == 1
                actual_negative = labels[j].cpu().numpy() == 0

                group_tprs[group].append(tp / (actual_positive + 1e-8))
                group_fprs[group].append(fp / (actual_negative + 1e-8))

    tpr_disparities = {}
    fpr_disparities = {}
    for group in group_tprs:
        tpr_disparities[group] = np.mean(group_tprs[group])
        fpr_disparities[group] = np.mean(group_fprs[group])

    for group in group_tprs:
        print(f"Group {group} TPR: {tpr_disparities[group]:.4f}, FPR: {fpr_disparities[group]:.4f}")
    
    return tpr_disparities, fpr_disparities

In [10]:
wga, group_accuracies = evaluate_worst_group_accuracy(model_before_mitigating, val_loader, group_labels)
dp_rates = evaluate_demographic_parity(model_before_mitigating, val_loader, group_labels)
eo_tprs = evaluate_equal_opportunity(model_before_mitigating, val_loader, group_labels)
tpr_disparities, fpr_disparities = evaluate_equalized_odds(model_before_mitigating, val_loader, group_labels)

Evaluating WGA: 100%|██████████| 127/127 [00:04<00:00, 30.48it/s]


Group 0 Accuracy: 0.2841
Group 1 Accuracy: 0.7168
Group 2 Accuracy: 0.9873
Group 3 Accuracy: 0.8784


Evaluating Demographic Parity: 100%|██████████| 127/127 [00:03<00:00, 33.60it/s]


Group 0 PPR: 0.7159
Group 1 PPR: 0.2832
Group 2 PPR: 0.9873
Group 3 PPR: 0.8784


Evaluating Equal Opportunity: 100%|██████████| 127/127 [00:03<00:00, 33.26it/s]


Group 0 TPR: 0.0000
Group 1 TPR: 0.0000
Group 2 TPR: 0.9873
Group 3 TPR: 0.8784


Evaluating Equalized Odds: 100%|██████████| 127/127 [00:03<00:00, 33.49it/s]

Group 0 TPR: 0.0000, FPR: 0.7159
Group 1 TPR: 0.0000, FPR: 0.2832
Group 2 TPR: 0.9873, FPR: 0.0000
Group 3 TPR: 0.8784, FPR: 0.0000


# Debiasing with D3M

In [12]:
print('YOYO')
ckpts = [model_before_mitigating.state_dict()]

dda = DDA(
    model=model_before_mitigating,
    checkpoints=[model_before_mitigating.state_dict()],
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    group_indices=group_inds
)

YOYO


Finalizing scores for all model IDs..: 100%|██████████| 1/1 [00:00<00:00,  1.48it/s]


In [14]:
# debiased_inds = dda.debias(use_heuristic=False, num_to_discard=100)
debiased_inds = dda.debias(use_heuristic=True)
len(debiased_inds)

13629

In [15]:
import copy

deep_copy_model = copy.deepcopy(model_before_mitigating)

# Machine Unlearning

In [16]:
all_train_indices = np.arange(len(train_loader.dataset))
harmful_indices = np.setdiff1d(all_train_indices, debiased_inds)

In [ ]:
def remove_influence(model, dataloader, harmful_indices, factor, device="cuda"):
    model.eval()
    harmful_dataset = torch.utils.data.Subset(dataloader.dataset, harmful_indices)
    harmful_loader = torch.utils.data.DataLoader(harmful_dataset, batch_size=1)

    for inputs, labels in harmful_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)

        loss = torch.nn.functional.cross_entropy(outputs, labels)
        grads = torch.autograd.grad(loss, model.parameters(), retain_graph=True)

        with torch.no_grad():
            for param, grad in zip(model.parameters(), grads):
                param -= grad * factor

    return model

results ={'factor':[], 'model':[], 'min':[], 'max':[], 'gap':[]}
factors = np.linspace(0.0001, 0.001, 2)

for factor in factors:
    newdeepmodel = copy.deepcopy(deep_copy_model)
    m = remove_influence(newdeepmodel, train_loader, harmful_indices, factor, device="cuda")
    wga, group_accs = evaluate_worst_group_accuracy(m, val_loader, group_inds, device="cuda")
    current_gap = (max(group_accs.values()) - wga)
    results['model'].append(m)
    results['min'].append(wga)
    results['max'].append(max(group_accs.values()))
    results['gap'].append(current_gap)
    results['factor'].append(factor)

In [36]:
import pandas as pd

df = pd.DataFrame(results).sort_values('factor')
df

,factor,model,min,max,gap
0,0.00001,"ResNet(\n (conv1): Conv2d(3, 64, kernel_size=...",0.327957,0.972888,0.644931
1,0.00010,"ResNet(\n (conv1): Conv2d(3, 64, kernel_size=...",0.419355,0.972888,0.553534


Now, it's time to investigate what are the best approaches to machine unlearning and how can we formulate that.

What are the other approaches to machine unlearning?

# Fair Pruning

In [ ]:
import torch
import copy

def fair_pruning(model, dataloader, harmful_indices, threshold=0.01, device="cuda"):
    model.eval()
    pruned_model = copy.deepcopy(model)
    harmful_dataset = torch.utils.data.Subset(dataloader.dataset, harmful_indices)
    harmful_loader = torch.utils.data.DataLoader(harmful_dataset, batch_size=1)

    parameter_gradients = []
    for inputs, labels in tqdm(harmful_loader, desc="Calculating Gradients"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = pruned_model(inputs)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        grads = torch.autograd.grad(loss, pruned_model.parameters(), retain_graph=True)
        parameter_gradients.append([grad.clone() for grad in grads])

    with torch.no_grad():
        for param, grads in zip(pruned_model.parameters(), zip(*parameter_gradients)):
            mean_grad = torch.mean(torch.stack(grads), dim=0)
            param[torch.abs(mean_grad) < threshold] = 0.0

    return pruned_model

pruned_model = fair_pruning(model_before_mitigating, train_loader, harmful_indices, threshold=0.01)

wga, group_accs = evaluate_worst_group_accuracy(pruned_model, val_loader, group_inds, device="cuda")

# Differentially Private Influence Functions for Unlearning

In [ ]:
import torch
import numpy as np
import copy

def dp_influence_unlearning(model, dataloader, harmful_indices, epsilon=1.0, delta=1e-5, device="cuda"):
    model.eval()
    updated_model = copy.deepcopy(model)
    harmful_dataset = torch.utils.data.Subset(dataloader.dataset, harmful_indices)
    harmful_loader = torch.utils.data.DataLoader(harmful_dataset, batch_size=1)

    sensitivity = 1.0 / len(harmful_loader)
    noise_scale = sensitivity * np.sqrt(2 * np.log(1.25 / delta)) / epsilon

    for inputs, labels in tqdm(harmful_loader, desc="Applying DP Influence Unlearning"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = updated_model(inputs)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        grads = torch.autograd.grad(loss, updated_model.parameters(), retain_graph=True)

        with torch.no_grad():
            for param, grad in zip(updated_model.parameters(), grads):
                noise = torch.normal(mean=0, std=noise_scale, size=grad.shape, device=device)
                param -= (grad + noise)

    return updated_model

dp_model = dp_influence_unlearning(model_before_mitigating, train_loader, harmful_indices, epsilon=1.0, delta=1e-5)
wga, group_accs = evaluate_worst_group_accuracy(dp_model, val_loader, group_inds, device="cuda")